# Finding out why Monte Carlo results are significantly different than static ones

Based on a [Stack Overflow question](https://stackoverflow.com/questions/72807629/overestimated-monte-carlo-results-in-brightway).

Uses Brightway 2.5.

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
import numpy as np
import pandas as pd

In [2]:
bd.projects.set_current("ecoinvent 3.8 consequential 25")

Define functional unit and LCIA impact category

In [3]:
waste_paper = bd.Database("ecoinvent 3.8 consequential").get(name='market for waste paper, sorted')
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

Create static and stochastic LCA instances

In [4]:
static = bc.LCA({waste_paper: 1}, ipcc)
static.lci()
static.lcia()

In [5]:
mc = bc.LCA({waste_paper: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

Confirm that the scores are quite different

In [6]:
static.score, np.mean([mc.score for _ in zip(range(20), mc)])

(2.76826441779881, 163.95582589488586)

We can do a recursive calculation on the Monte Carlo object (this doesn't call `__next__`, so the matrices remain the same), but we need to give some normally private arguments:

In [7]:
ba.print_recursive_calculation(
    waste_paper,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.01,
    __lca_obj=mc,
    __total_score=mc.score,
)

Fraction of score | Absolute score | Amount | Activity
0001 |  47.2 |     1 | 'market for waste paper, sorted' (kilogram, GLO, None)
  0.288 | 13.61 | 0.2857 | 'tissue paper production' (kilogram, RER, None)
    0.296 | 13.96 | 0.2434 | 'market for tissue paper' (kilogram, GLO, None)
      0.295 | 13.94 | 0.2434 | 'tissue paper production, virgin' (kilogram, GLO, None)
        0.0127 | 0.5986 | 0.247 | 'market for sulfite pulp, bleached' (kilogram, GLO, None)
  0.712 | 33.59 | 0.7143 | 'tissue paper production' (kilogram, RoW, None)
    0.74 |  34.9 | 0.6084 | 'market for tissue paper' (kilogram, GLO, None)
      0.739 | 34.86 | 0.6084 | 'tissue paper production, virgin' (kilogram, GLO, None)
        0.0146 | 0.6883 | 4.327 | 'market group for heat, district or industrial, other than natural gas
          0.0123 | 0.5797 | 3.586 | 'market for heat, district or industrial, other than natural gas' (meg
        0.0178 | 0.8388 | 1.217 | 'market group for electricity, medium voltage' (kilo

We can see from this that there is a large impact from `tissue paper production, virgin`. Let's switch our functional unit to this:

In [8]:
tissue = bd.Database("ecoinvent 3.8 consequential").get(name='tissue paper production, virgin')

But when we try to look through the supply chain to understand why we get such high stochastic impacts, we don't get anywhere:

In [9]:
static = bc.LCA({tissue: 1}, ipcc)
static.lci()
static.lcia()

In [10]:
mc = bc.LCA({tissue: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

In [11]:
ba.print_recursive_calculation(
    tissue,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.025,
    __lca_obj=mc,
    __total_score=mc.score,
)

Fraction of score | Absolute score | Amount | Activity
0001 | 149.5 |     1 | 'tissue paper production, virgin' (kilogram, GLO, None)


We seem to be missing something important - the total (stochastic) score is ~114, but the only input above our 2.5% cutoff is sulfite pulp, which is only 3.6% of the total score. Maybe this activity has some high direct emissions?

In [12]:
mc.characterized_inventory[:, mc.dicts.activity[tissue.id]].sum()

0.0

We don't see anything in our supply chain, but we can see which activities generate actual impacts by doing a contribution analysis on the final characterized inventory. However, our recursive calculation changed the functional unit while recursing, so we need to "reset" the calculation:

In [13]:
mc = bc.LCA({tissue: 1}, ipcc, use_distributions=True)
mc.lci()
mc.lcia()

In [14]:
pd.DataFrame(
    [
        (x, z['name'], z['location']) 
        for x,_,z in ba.ContributionAnalysis().annotated_top_processes(mc)
    ], 
    columns=['Score', 'Name', 'Location']
)

,Score,Name,Location
0,20.286712,palm fruit bunch production,MY
1,13.118425,"heat and power co-generation, lignite",RoW
2,8.644094,"heat and power co-generation, hard coal",RU
3,5.797260,"clear-cutting, secondary forest to arable land...",MY
4,5.715522,"heat and power co-generation, natural gas, 200...",RoW
5,4.270209,"heat and power co-generation, hard coal",PL
6,3.215141,"coconut production, dehusked",RoW
7,2.619751,"heat and power co-generation, lignite",PL
8,2.556957,"heat and power co-generation, lignite",RU
9,2.522933,"clear-cutting, primary forest to arable land, ...",ID


[Something is hapening here, but you don't know what it is](https://www.youtube.com/watch?v=we37yX3zpKA)...

Not going to lie, this was very confusing. There are no direct emissions from tissue paper, but also no impacts from the supply chain of tissue paper. Eventually I realized that ew can compare the values in the technosphere matrix for the static and Monte Carlo cases and see how different the input amounts are:

In [15]:
def compare_inputs_two_lca_objects(first, second, column_id):
    column_index = first.dicts.activity[column_id]
    results = []
    
    for row in first.technosphere_matrix[:, column_index].tocoo().row:
        a = abs(first.technosphere_matrix[row, column_index])
        b = abs(second.technosphere_matrix[row, column_index])
        try:
            ratio = a / b
        except ZeroDivisionError:
            ratio = 0
        if ratio > 10 or ratio < 0.1:
            act = bd.get_activity(first.dicts.product.reversed[row])
            results.append({
                'ratio': ratio, 
                'a_value': a,
                'b_value': b,
                'name': act['name'],
                'location': act['name'],
                'act_id': act.id,
            })
            
    if results:
        results.sort(key=lambda x: x['ratio'], reverse=True)
        results = pd.DataFrame(results)
    return results

In [16]:
compare_inputs_two_lca_objects(mc, static, tissue.id)

,ratio,a_value,b_value,name,location,act_id
0,1726.007803,17.260078,0.010000,"market for chemical, inorganic","market for chemical, inorganic",14207
1,1683.925576,1.683926,0.001000,"market for chemical, organic","market for chemical, organic",5475
2,1346.659786,17.420700,0.012936,"market for rosin size, for paper production","market for rosin size, for paper production",8907
3,1299.454719,35.168144,0.027064,"market for rosin size, for paper production","market for rosin size, for paper production",6383
4,0.057935,0.001086,0.018750,market for potato starch,market for potato starch,21449


We are finally getting somewhere! We have four inputs to the tissue paper where the static amounts are **much less** than the stochastic values sampled. Let's look at one of the uncertainty distributions ([here is the activity dataset at ecoinvent](https://v38.ecoquery.ecoinvent.org/Details/UPR/62960c80-c028-44d4-9d45-a4a6a7a689c6/dd7f13f5-0658-489c-a19c-f2ff8a00bdd9)):

In [17]:
exc = next(exc for exc in tissue.technosphere() if exc.input.id == 8907)
dist = {attr: exc.get(attr) for attr in ('uncertainty type', 'minimum', 'loc', 'maximum')}
dist

{'uncertainty type': 5,
 'minimum': 0.0,
 'loc': 0.0129362298845668,
 'maximum': 80.0}

`5` is the [triangular distribution](https://en.wikipedia.org/wiki/Triangular_distribution), this comes from [stats_arrays](https://stats-arrays.readthedocs.io/en/latest/).

So, what does this mean? Let's look at the statistics:

In [18]:
import stats_arrays as sa

In [19]:
sa.TriangularUncertainty.statistics(sa.TriangularUncertainty.from_dicts(dist))

{'mean': 26.670978743294857,
 'median': 23.436031337921335,
 'mode': 0.0129362298845668,
 'lower': 0.5080001645503973,
 'upper': 71.05645127647223}

When the mean is more than 200 times the static value used, it makes sense that the Monte Carlo results would be significantly biased. And we see a similar pattern in the other three exchanges with high ratios.

Why didn't we see this in our recursive graph traversal? Because that uses the static exchange values instead of the values in the matrix itself:

```python
for exc in activity.technosphere():
```

We can change the function to use the values in the technosphere:

In [20]:
import sys

def print_recursive_calculation_sample_values(
    activity,
    lcia_method,
    amount=1,
    max_level=3,
    cutoff=1e-2,
    file_obj=None,
    tab_character="  ",
    __level=0,
    __lca_obj=None,
    __total_score=None,
    __first=True,
):
    """Traverse a supply chain graph, and calculate the LCA scores of each component. Prints the result with the format:

    {tab_character * level }{fraction of total score} ({absolute LCA score for this input} | {amount of input}) {input activity}

    Args:
        activity: ``Activity``. The starting point of the supply chain graph.
        lcia_method: tuple. LCIA method to use when traversing supply chain graph.
        amount: int. Amount of ``activity`` to assess.
        max_level: int. Maximum depth to traverse.
        cutoff: float. Fraction of total score to use as cutoff when deciding whether to traverse deeper.
        file_obj: File-like object (supports ``.write``), optional. Output will be written to this object if provided.
        tab_character: str. Character to use to indicate indentation.

    Internal args (used during recursion, do not touch);
        __level: int.
        __lca_obj: ``LCA``.
        __total_score: float.
        __first: bool.

    Returns:
        Nothing. Prints to ``sys.stdout`` or ``file_obj``

    """
    activity = bd.get_activity(activity)
    if file_obj is None:
        file_obj = sys.stdout

    if __lca_obj is None:
        __lca_obj = bc.LCA({activity: amount}, lcia_method)
        __lca_obj.lci()
        __lca_obj.lcia()
        __total_score = __lca_obj.score
    elif __total_score is None:
        raise ValueError
    else:
        __lca_obj.redo_lcia({activity.id: amount})
        if abs(__lca_obj.score) <= abs(__total_score * cutoff):
            return
    if __first:
        file_obj.write("Fraction of score | Absolute score | Amount | Activity\n")
    message = "{}{:04.3g} | {:5.4n} | {:5.4n} | {:.70}".format(
        tab_character * __level,
        __lca_obj.score / __total_score,
        __lca_obj.score,
        float(amount),
        str(activity),
    )
    file_obj.write(message + "\n")
    if __level < max_level:
        prod_exchanges = list(activity.production())
        if not prod_exchanges:
            prod_amount = 1
        elif len(prod_exchanges) > 1:
            warn("Hit multiple production exchanges; aborting in this branch")
            return
        else:
            prod_amount = __lca_obj.technosphere_matrix[
                __lca_obj.dicts.product[prod_exchanges[0].input.id],
                __lca_obj.dicts.activity[prod_exchanges[0].output.id],
            ]

        for exc in activity.technosphere():
            if exc.input.id == exc.output.id:
                continue

            # Everything except this section stays the same
            # <<<
            sign = -1 if exc.get('type') in ('technosphere', 'generic technosphere') else 1
            tm_amount = __lca_obj.technosphere_matrix[
                __lca_obj.dicts.product[exc.input.id],
                __lca_obj.dicts.activity[exc.output.id],
            ] * sign
            # >>>
            print_recursive_calculation_sample_values(
                activity=exc.input,
                lcia_method=lcia_method,
                amount=amount * tm_amount / prod_amount,  # And here we use `tm_amount` instead of exc['amount']
                max_level=max_level,
                cutoff=cutoff,
                file_obj=file_obj,
                tab_character=tab_character,
                __first=False,
                __lca_obj=__lca_obj,
                __total_score=__total_score,
                __level=__level + 1,
            )

In [21]:
print_recursive_calculation_sample_values(
    tissue,
    ipcc,
    amount=1,
    max_level=5,
    cutoff=0.025,
    __lca_obj=mc,
    __total_score=mc.score,
)

Fraction of score | Absolute score | Amount | Activity
0001 |   134 |     1 | 'tissue paper production, virgin' (kilogram, GLO, None)
  0.0297 | 3.984 | 1.684 | 'market for chemical, organic' (kilogram, GLO, None)
    0.0297 | 3.984 | 1.684 | 'chemical production, organic' (kilogram, GLO, None)
  0.258 | 34.64 | 17.26 | 'market for chemical, inorganic' (kilogram, GLO, None)
    0.258 | 34.64 | 17.26 | 'chemical production, inorganic' (kilogram, GLO, None)
      0.026 | 3.489 | 0.4632 | 'market for titanium dioxide' (kilogram, RoW, None)
      0.0322 | 4.313 | 1.287 | 'market for sodium chlorate, powder' (kilogram, RoW, None)
        0.0314 | 4.212 | 1.284 | 'sodium chlorate production, powder' (kilogram, RoW, None)
  0.278 | 37.31 | 17.42 | 'market for rosin size, for paper production' (kilogram, RER, None)
    0.273 | 36.65 | 17.42 | 'rosin size production, for paper production' (kilogram, RER, None)
      0.0318 | 4.258 | 2.494 | 'market for sodium hydroxide, without water, in 50% so

We can now find interesting results more quickly. This function will be updated in `bw2analyzer` real soon now™.